## Sources:
https://github.com/maxbozza/HOG-SVMlight-trainer/blob/master/train_SVMlight.py

https://github.com/aarcosg/object-detector-svm-hog-python


In [ ]:
import 

## 1)
Harris köşe bulma algoritmasını (k = 0.05, W = 7x7 ve köşelilik değerleri üzerinde uygun bir eşik  değeri)ile  çalıştırınız

Köşe  bulma  algoritmasını  çalıştırmadan  evvel  gerek görülürse arka plan bilgisini standarda oturtmak için bir ön işlemedüşünülebilir.Harris köşe bulma algoritması farklı ölçeklerde hesaplanıp yerel maksimum noktalarının belirlenmesiyle köşe noktaları ölçekten bağımsız hale getirilebilir.

## 2) 
Belirlenen bu köşe noktalarıiçin  tanımlayıcılar  oluşturulacaktır. Tanımlayıcılar  eğimlerin  histogramı  (Histogram  of  gradients-HoG)  yöntemi  kullanılarak hesaplanacaktır.  Bunun  için  bulunan  ilgi  noktası  koordinatlarını  merkeze  alan W=32x32boyutunda pencerelerde çalışılabilir1. Burada Wtoplamda 4x4‘lük 16hücreye ayrılmaktadır.Bu durumda her bir hücre 8x8 pikselden oluşur.Ardından her bir hücre içerisindeki pikseller için; açı bilgisi histogram kovasının indeksi; genlik bilgisi ise bu kovalara atanan değerler olarak  kullanılıp  eğimlerin  histogramı  oluşturulacaktır.  Bu  histogramlar  ardından  genlik değerleri  toplamı  ile  normalize  edilecektir.  Böylece toplam  16  hücrede histogramın  kova sayısı  8  olacak  şekilde  açı  aralıkları  belirlendiğinde  her  bir ilgi noktasıiçin  16x8  =  128 uzunluğunda bir tanımlayıcı bulunacaktır2. Not:Burada farklı renk kanallarına ait genlik ve açı bilgilerinden en yüksek olanının seçilmesiyle kanal sayısı bire indirgenebilir.

## 3) 
Bulunan tanımlayıcılarilgili sınıfa ait tüm eğitim görüntüleri için uygun etiketlerle bir kenara kaydedilip gerektiğindetest görüntülerini sınıflandırma amacıyla kullanılacaktır. Bunun içintüm test görüntülerinde de benzertanımlayıcılar çıkarılacaktır. Bir test görüntüsüne ait her bir tanımlayıcı için k-en yakın komşu algoritmasıyla eğitimde belirlenen tanımlayıcı kümelerine olan benzerlik yüzdeleri hesaplanacaktır. k değeri burada 3, 5veya 7olarak denenebilir. Test görüntüsü tüm tanımlayıcılareşlendikten  sonra en  yüksek  yüzdeyi  veren  sınıfın  etiketiyle etiketlenecektir.

## 4) 
Bulunan tanımlayıcılar ilgili sınıfa ait tüm eğitim görüntüleri için uygun etiketlerle bir kenara kaydedilip gerektiğindetest görüntülerini sınıflandırma amacıyla kullanılacaktır. Bunun içintüm test görüntülerinde de benzertanımlayıcılar çıkarılacaktır. Bir test görüntüsüne ait her bir tanımlayıcı için k-en yakın komşu algoritmasıyla eğitimde belirlenen tanımlayıcı kümelerine olan benzerlik yüzdeleri hesaplanacaktır. k değeri burada 3, 5veya 7olarak denenebilir. Test görüntüsü tüm tanımlayıcılareşlendikten  sonra en  yüksek  yüzdeyi  veren  sınıfın  etiketiyle etiketlenecektir.

## Eğitim:
Her bir sınıf için,1.Eğitim görüntüleri içerisinde Harris köşe noktalarını belirle ve her nokta için HoG tanımlayıcılarını hesapla. 2.Bu tanımlayıcıları ilgili sınıfıtemsil etmek için bir dosyada veya veri yapısında sakla.

## Test:
    Test kümesindeki her bir görüntü için,
        * 1. Harris köşe noktalarını belirle ve bunların HoG tanımlayıcılarını hesapla.  
        * 2. Her  bir  tanımlayıcının  eğitim  sürecinde  kaydedilen  farklı  sınıflara  ait tanımlayıcılara  olan  mesafesini hesapla.    
            En  düşük  mesafeye  sahip  k tanımlayıcıyı  belirledikten  sonra  ilgili özelliğin hangi  sınıfı  temsil  ediyor olduğuna karar ver. 
            Buna göre ilgili sınıfa ait olma sayacını bir arttır.   
    Not: Eğer bir özelliğin en yakın eşlemeleri çok farklı sınıflara ait oluyorsa bu durum  belirsizlik  (ambiguity)  olarak  değerlendirilip  ilgili  özellik  dikkate alınmayabilir.Alternatif   olarak   ken  yakın  komşu  içerisinde  hangi  sınıf etiketleri oluşmuşsa bu etiketler test örneğinin ilgili sınıfa ait olma olasılığını arttırmada kullanılabilir.3.Nihayette  ilgili  test görüntüsünün hangi sınıfa ait olma olasılığı daha yüksek ise görüntüyü o sınıfın etiketiyle etiketlendir.